In [1]:
import requests
from bs4 import BeautifulSoup

base_url = "https://pandas.pydata.org/docs/reference/index.html"
response = requests.get(base_url)

In [2]:
soup = BeautifulSoup(response.text, "lxml")
navbar = soup.find_all(attrs={"class": "reference internal"})

In [3]:
ref_url = "https://pandas.pydata.org/docs/reference/"
ref_url+ navbar[0]['href']

'https://pandas.pydata.org/docs/reference/io.html'

In [4]:
first_level = {}
for nbar in navbar:
    text_val = nbar.text
    href = nbar['href']
    full_url = ref_url+ href
    if  "#" not in href:
        first_level.update({href:{"functions":[],"name":text_val,"url":full_url}})
    else:
        break

In [5]:
for nbar in navbar:
    text_val = nbar.text
    href = nbar['href']
    full_url = ref_url+ href
    if "#" in href:
        parent_html = href.split("#")[0]
        if parent_html in first_level:
            first_level[parent_html]['functions'].append({"name":text_val,"url":full_url})
        else:
            print(text_val)

In [6]:
first_level['io.html']['url']
# https://pandas.pydata.org/docs/reference/io.html

'https://pandas.pydata.org/docs/reference/io.html'

## EXTRACTING FUNCTIONS

In [7]:
# parent_url = first_level['io.html']['url']

# parent_soup = BeautifulSoup(requests.get(parent_url).text,'lxml')
# url_id = "pickling"
# s1 = parent_soup.find(attrs={"id":url_id})
# s1

In [8]:
base_func_url = "https://pandas.pydata.org/docs/reference/"
for first_level_name in first_level:
    parent_url = first_level[first_level_name]['url']
    parent_soup = BeautifulSoup(requests.get(parent_url).text,'lxml')
    all_urls = []
    for func in first_level[first_level_name]['functions']:
        url_id = func['url'].split("/")[-1].split("#")[-1]
        try:
            func_url = parent_soup.find(attrs={"id":url_id}).find(attrs={"class":"row-odd"}).find('a')['href']
            all_urls.append(base_func_url+func_url)
        except Exception as e:
            print(e)
            print(first_level_name,url_id)
    first_level[first_level_name]['function_urls'] = all_urls

'NoneType' object is not subscriptable
arrays.html objects


In [34]:
from tqdm import tqdm
def get_param_data(first_level):
    for parent, values in first_level.items():
        for func_url in tqdm(values['function_urls']):
            func_response = requests.get(func_url)
            func_soup = BeautifulSoup(func_response.text, "lxml")
            
            func_name = func_soup.find("h1").text.replace("#","") #remove #
            try:
                elem = func_soup.find(attrs={"class":"sig sig-object py"})
                full_function = elem.text.replace("[source]#","").replace("\n","")
                func_text = func_soup.find("dd").find('p').text
                em = func_soup.find_all(attrs={"class":"field-odd"})
                param_names = em[-1].find_all("dt")
                desc_list = em[-1].find_all("dd")

                if len(param_names)!=len(desc_list):
                    print(func_name)
                    print(f"param_names={len(param_names)} and desc_list={len(desc_list)}")
            except Exception as e:
                print(e)
                print(func_name,func_text)

 93%|█████████▎| 14/15 [00:01<00:00,  7.47it/s]

list index out of range
pandas.Series.str Vectorized string functions for Series and Index.


 82%|████████▏ | 14/17 [00:01<00:00,  8.02it/s]

list index out of range
pandas.DataFrame.attrs Dictionary of global attributes of this dataset.


100%|██████████| 17/17 [00:02<00:00,  7.10it/s]


...

 60%|██████    | 3/5 [00:00<00:00,  8.19it/s]

'NoneType' object has no attribute 'text'
pandas.io.formats.style.Styler.env Helps style a DataFrame or Series according to the data with HTML and CSS.


100%|██████████| 5/5 [00:00<00:00,  7.14it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  8.83it/s]
0it [00:00, ?it/s]
 75%|███████▌  | 3/4 [00:00<00:00,  7.31it/s]

list index out of range
pandas.errors.AbstractMethodError Raise this error instead of NotImplementedError for abstract methods.


100%|██████████| 4/4 [00:00<00:00,  7.38it/s]
0it [00:00, ?it/s]


In [35]:
func_url = 'https://pandas.pydata.org/docs/reference/api/pandas.read_clipboard.html'
func_url = 'https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.nanoseconds.html'
func_response = requests.get(func_url)
func_soup = BeautifulSoup(func_response.content, "lxml",from_encoding="utf-8")

In [36]:
func_name = func_soup.find("h1").text.replace("#","") #remove #

In [37]:
func_name

'pandas.Series.dt.nanoseconds'

In [38]:
elem = func_soup.find(attrs={"class":"sig sig-object py"})
full_function = elem.text.replace("[source]#","").replace("\n","")
func_text = func_soup.find("dd").find('p').text

In [39]:
func_text

'Number of nanoseconds (>= 0 and less than 1 microsecond) for each element.'

In [40]:
param_names = func_soup.find(attrs={"class":"field-list"})

In [46]:
em1 = param_names.find_all(attrs={"class":"field-odd"})

AttributeError: 'NoneType' object has no attribute 'find_all'

In [44]:
param_names

In [48]:
# len(em[-1].find_all('dt'))
em[-1].find_all('dt')

[<dt><strong>extra_args</strong><span class="classifier">list[str], default None</span></dt>,
 <dt><strong>run_doctests</strong><span class="classifier">bool, default False</span></dt>]

In [49]:
import re
def remove_non_english(sentence):
    # Define a regular expression pattern to match non-English characters
    non_english_pattern = re.compile(r'[^a-zA-Z0-9\s,.!?}(){]+\\')
    
    # Replace non-English characters with an empty string
    cleaned_sentence = non_english_pattern.sub('', sentence)
    # cleaned_sentence = re.sub('â','',sentence)
    # cleaned_sentence = re.sub('â','',sentence)
    return cleaned_sentence
for pname,param_desc in zip(em[-1].find_all('dt'),em[-1].find_all('dd')):
    # print('-'*100)
    param_name_text = pname.text
    desc_text = param_desc.text
    if "**kwargs" in param_name_text: continue
    process_param_name = remove_non_english(param_name_text)
    process_desc_text = remove_non_english(desc_text)
    # print(process_param_name)
    # print(process_desc_text)
    print(pname.text.strip())
    print(param_desc.text.strip())
    print('-'*100)

extra_argslist[str], default None
Extra marks to run the tests.
----------------------------------------------------------------------------------------------------
run_doctestsbool, default False
Whether to only run the Python and Cython doctests. If you would like to run
both doctests/regular tests, just append ââdoctest-modulesâ/ââdoctest-cythonâ
to extra_args.
----------------------------------------------------------------------------------------------------


In [19]:
func_soup.find(attrs={"class":"field-list"}).find(attrs={"class":"classifier"})

<span class="classifier">str, default â\s+â</span>

In [20]:
em = func_soup.find_all(attrs={"class":"field-odd"})

In [21]:
em[-1].find_all("dt")

[<dt><strong>sep</strong><span class="classifier">str, default â\s+â</span></dt>,
 <dt><strong>dtype_backend</strong><span class="classifier">{ânumpy_nullableâ, âpyarrowâ}, default ânumpy_nullableâ</span></dt>,
 <dt><strong>**kwargs</strong></dt>]

In [22]:
for p in em[-1].find_all("p"):

    if p.find(attrs={"class":"versionmodified added"}) is None and p.find(attrs={"class":"versionmodified changed"}) is None:
        print(p.text)
        print('-'*100)

A string or regex delimiter. The default of '\\s+' denotes
one or more whitespace characters.
----------------------------------------------------------------------------------------------------
Back-end data type applied to the resultant DataFrame
(still experimental). Behaviour is as follows:
----------------------------------------------------------------------------------------------------
"numpy_nullable": returns nullable-dtype-backed DataFrame
(default).
----------------------------------------------------------------------------------------------------
"pyarrow": returns pyarrow-backed nullable ArrowDtype
DataFrame.
----------------------------------------------------------------------------------------------------
See read_csv() for the full argument list.
----------------------------------------------------------------------------------------------------
